### 1. Import needed packages and clone the dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from sklearn.datasets import load_files
from keras.utils import np_utils, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

print('You are using numpy version:', np.__version__)

!git clone https://github.com/Horea94/Fruit-Images-Dataset

train_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Training/'
test_dir = 'C:/Users/crysi/Private Dokumente/Papers & wissenschaftl. Arbeiten/Master/Foundations of Machine Learning/First Project/Pycharm/Fruit-Images-Dataset/Test/'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files, targets, target_labels

names_train, intclass_train, stringclass_train = load_dataset(train_dir)
names_test, intclass_test, stringclass_test = load_dataset(test_dir)

n_classes = len(np.unique(intclass_test))

train_hot_class = np_utils.to_categorical(intclass_train, n_classes)
test_hot_class = np_utils.to_categorical(intclass_test, n_classes)

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

train_images_array = np.array(convert_image_to_array(names_train))
test_images_array = np.array(convert_image_to_array(names_test))

train_images_array = train_images_array.astype('float32')/255
test_images_array = test_images_array.astype('float32')/255

from sklearn.model_selection import train_test_split
#split training into another training and test (to reduce computation resources/time)
train_images_array, new_test_images_array, train_hot_class, new_test_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.3, random_state = 42)

train_images_array, valid_images_array, train_hot_class, valid_hot_class = train_test_split(train_images_array, train_hot_class, test_size = 0.2)

You are using numpy version: 1.20.1


fatal: destination path 'Fruit-Images-Dataset' already exists and is not an empty directory.


In [21]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [22]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 34s - loss: 0.0171 - accuracy: 0.6526 - val_loss: 0.0159 - val_accuracy: 0.6688 - 34s/epoch - 29ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0113 - accuracy: 0.8432 - val_loss: 0.0141 - val_accuracy: 0.7468 - 10s/epoch - 8ms/step
Epoch 3/100
1185/1185 - 8s - loss: 0.0102 - accuracy: 0.8903 - val_loss: 0.0095 - val_accuracy: 0.9177 - 8s/epoch - 6ms/step
Epoch 4/100
1185/1185 - 8s - loss: 0.0097 - accuracy: 0.9071 - val_loss: 0.0090 - val_accuracy: 0.9392 - 8s/epoch - 6ms/step
Epoch 5/100
1185/1185 - 7s - loss: 0.0094 - accuracy: 0.9233 - val_loss: 0.0093 - val_accuracy: 0.9254 - 7s/epoch - 6ms/step
Epoch 6/100
1185/1185 - 7s - loss: 0.0092 - accuracy: 0.9323 - val_loss: 0.0117 - val_accuracy: 0.8449 - 7s/epoch - 6ms/step
Epoch 7/100
1185/1185 - 7s - loss: 0.0090 - accuracy: 0.9416 - val_loss: 0.0098 - val_accuracy: 0.8993 - 7s/epoch - 6ms/step
Epoch 8/100
1185/1185 - 7s - loss: 0.0088 - accuracy: 0.9500 - val_loss: 0.0096 - val_accuracy: 0

In [23]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [24]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 11s - loss: 0.0161 - accuracy: 0.7081 - val_loss: 0.0155 - val_accuracy: 0.6755 - 11s/epoch - 9ms/step
Epoch 2/100
1185/1185 - 7s - loss: 0.0105 - accuracy: 0.8863 - val_loss: 0.0141 - val_accuracy: 0.7386 - 7s/epoch - 6ms/step
Epoch 3/100
1185/1185 - 7s - loss: 0.0096 - accuracy: 0.9182 - val_loss: 0.0125 - val_accuracy: 0.7971 - 7s/epoch - 6ms/step
Epoch 4/100
1185/1185 - 7s - loss: 0.0093 - accuracy: 0.9304 - val_loss: 0.0095 - val_accuracy: 0.9216 - 7s/epoch - 6ms/step
Epoch 5/100
1185/1185 - 7s - loss: 0.0090 - accuracy: 0.9421 - val_loss: 0.0097 - val_accuracy: 0.9098 - 7s/epoch - 6ms/step
Epoch 6/100
1185/1185 - 7s - loss: 0.0088 - accuracy: 0.9485 - val_loss: 0.0101 - val_accuracy: 0.8888 - 7s/epoch - 6ms/step
Epoch 7/100
1185/1185 - 7s - loss: 0.0087 - accuracy: 0.9546 - val_loss: 0.0090 - val_accuracy: 0.9392 - 7s/epoch - 6ms/step
Epoch 8/100
1185/1185 - 7s - loss: 0.0086 - accuracy: 0.9598 - val_loss: 0.0083 - val_accuracy: 0.97

In [25]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    #model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [26]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 20s - loss: 0.0223 - accuracy: 0.6057 - val_loss: 0.0152 - val_accuracy: 0.7512 - 20s/epoch - 17ms/step
Epoch 2/100
1185/1185 - 8s - loss: 0.0118 - accuracy: 0.8766 - val_loss: 0.0120 - val_accuracy: 0.8363 - 8s/epoch - 7ms/step
Epoch 3/100
1185/1185 - 8s - loss: 0.0101 - accuracy: 0.9150 - val_loss: 0.0162 - val_accuracy: 0.6500 - 8s/epoch - 7ms/step
Epoch 4/100
1185/1185 - 9s - loss: 0.0094 - accuracy: 0.9382 - val_loss: 0.0128 - val_accuracy: 0.7861 - 9s/epoch - 7ms/step
Epoch 5/100
1185/1185 - 10s - loss: 0.0091 - accuracy: 0.9457 - val_loss: 0.0119 - val_accuracy: 0.8067 - 10s/epoch - 8ms/step
Epoch 6/100
1185/1185 - 9s - loss: 0.0089 - accuracy: 0.9482 - val_loss: 0.0092 - val_accuracy: 0.9394 - 9s/epoch - 7ms/step
Epoch 7/100
1185/1185 - 8s - loss: 0.0087 - accuracy: 0.9591 - val_loss: 0.0110 - val_accuracy: 0.8557 - 8s/epoch - 7ms/step
Epoch 8/100
1185/1185 - 8s - loss: 0.0086 - accuracy: 0.9625 - val_loss: 0.0120 - val_accuracy: 0

In [27]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.5))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [28]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 11s - loss: 0.0325 - accuracy: 0.1717 - val_loss: 0.0244 - val_accuracy: 0.3201 - 11s/epoch - 10ms/step
Epoch 2/100
1185/1185 - 9s - loss: 0.0227 - accuracy: 0.4010 - val_loss: 0.0174 - val_accuracy: 0.5846 - 9s/epoch - 8ms/step
Epoch 3/100
1185/1185 - 9s - loss: 0.0197 - accuracy: 0.5045 - val_loss: 0.0171 - val_accuracy: 0.6154 - 9s/epoch - 8ms/step
Epoch 4/100
1185/1185 - 9s - loss: 0.0181 - accuracy: 0.5656 - val_loss: 0.0119 - val_accuracy: 0.8434 - 9s/epoch - 8ms/step
Epoch 5/100
1185/1185 - 9s - loss: 0.0172 - accuracy: 0.6001 - val_loss: 0.0153 - val_accuracy: 0.6614 - 9s/epoch - 7ms/step
Epoch 6/100
1185/1185 - 9s - loss: 0.0166 - accuracy: 0.6271 - val_loss: 0.0119 - val_accuracy: 0.8383 - 9s/epoch - 7ms/step
Epoch 7/100
1185/1185 - 9s - loss: 0.0160 - accuracy: 0.6544 - val_loss: 0.0113 - val_accuracy: 0.8590 - 9s/epoch - 7ms/step
Epoch 8/100
1185/1185 - 9s - loss: 0.0156 - accuracy: 0.6665 - val_loss: 0.0112 - val_accuracy: 0.8

In [29]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.5))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [30]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 19s - loss: 0.0262 - accuracy: 0.3411 - val_loss: 0.0249 - val_accuracy: 0.3411 - 19s/epoch - 16ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0171 - accuracy: 0.6186 - val_loss: 0.0143 - val_accuracy: 0.7431 - 10s/epoch - 8ms/step
Epoch 3/100
1185/1185 - 11s - loss: 0.0151 - accuracy: 0.6966 - val_loss: 0.0121 - val_accuracy: 0.8150 - 11s/epoch - 9ms/step
Epoch 4/100
1185/1185 - 11s - loss: 0.0140 - accuracy: 0.7384 - val_loss: 0.0161 - val_accuracy: 0.6769 - 11s/epoch - 9ms/step
Epoch 5/100
1185/1185 - 10s - loss: 0.0134 - accuracy: 0.7651 - val_loss: 0.0141 - val_accuracy: 0.7307 - 10s/epoch - 8ms/step
Epoch 6/100
1185/1185 - 10s - loss: 0.0130 - accuracy: 0.7769 - val_loss: 0.0174 - val_accuracy: 0.6485 - 10s/epoch - 8ms/step
Epoch 7/100
1185/1185 - 10s - loss: 0.0125 - accuracy: 0.7953 - val_loss: 0.0136 - val_accuracy: 0.7766 - 10s/epoch - 8ms/step
Epoch 8/100
1185/1185 - 10s - loss: 0.0124 - accuracy: 0.8010 - val_loss: 0.0149 - val_

In [31]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.5))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [32]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 14s - loss: 0.0390 - accuracy: 0.0694 - val_loss: 0.0310 - val_accuracy: 0.1820 - 14s/epoch - 12ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0282 - accuracy: 0.2349 - val_loss: 0.0207 - val_accuracy: 0.5633 - 10s/epoch - 9ms/step
Epoch 3/100
1185/1185 - 10s - loss: 0.0239 - accuracy: 0.3463 - val_loss: 0.0189 - val_accuracy: 0.5796 - 10s/epoch - 8ms/step
Epoch 4/100
1185/1185 - 10s - loss: 0.0218 - accuracy: 0.4170 - val_loss: 0.0165 - val_accuracy: 0.6615 - 10s/epoch - 9ms/step
Epoch 5/100
1185/1185 - 10s - loss: 0.0203 - accuracy: 0.4689 - val_loss: 0.0145 - val_accuracy: 0.7425 - 10s/epoch - 9ms/step
Epoch 6/100
1185/1185 - 11s - loss: 0.0190 - accuracy: 0.5175 - val_loss: 0.0140 - val_accuracy: 0.7694 - 11s/epoch - 9ms/step
Epoch 7/100
1185/1185 - 11s - loss: 0.0181 - accuracy: 0.5546 - val_loss: 0.0138 - val_accuracy: 0.7315 - 11s/epoch - 9ms/step
Epoch 8/100
1185/1185 - 10s - loss: 0.0174 - accuracy: 0.5838 - val_loss: 0.0118 - val_

In [33]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))
    model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [34]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 16s - loss: 0.0230 - accuracy: 0.4550 - val_loss: 0.0168 - val_accuracy: 0.6140 - 16s/epoch - 13ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0150 - accuracy: 0.7019 - val_loss: 0.0164 - val_accuracy: 0.6445 - 10s/epoch - 9ms/step
Epoch 3/100
1185/1185 - 10s - loss: 0.0133 - accuracy: 0.7642 - val_loss: 0.0129 - val_accuracy: 0.7750 - 10s/epoch - 9ms/step
Epoch 4/100
1185/1185 - 10s - loss: 0.0124 - accuracy: 0.7959 - val_loss: 0.0094 - val_accuracy: 0.9271 - 10s/epoch - 8ms/step
Epoch 5/100
1185/1185 - 10s - loss: 0.0118 - accuracy: 0.8221 - val_loss: 0.0110 - val_accuracy: 0.8559 - 10s/epoch - 8ms/step
Epoch 6/100
1185/1185 - 10s - loss: 0.0114 - accuracy: 0.8397 - val_loss: 0.0122 - val_accuracy: 0.8016 - 10s/epoch - 8ms/step
Epoch 7/100
1185/1185 - 10s - loss: 0.0111 - accuracy: 0.8508 - val_loss: 0.0113 - val_accuracy: 0.8483 - 10s/epoch - 8ms/step
Epoch 8/100
1185/1185 - 10s - loss: 0.0109 - accuracy: 0.8578 - val_loss: 0.0104 - val_

In [35]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='tanh'))
    model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [36]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 18s - loss: 0.0194 - accuracy: 0.5897 - val_loss: 0.0185 - val_accuracy: 0.5656 - 18s/epoch - 15ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0127 - accuracy: 0.7989 - val_loss: 0.0169 - val_accuracy: 0.6441 - 10s/epoch - 9ms/step
Epoch 3/100
1185/1185 - 11s - loss: 0.0113 - accuracy: 0.8451 - val_loss: 0.0128 - val_accuracy: 0.7950 - 11s/epoch - 9ms/step
Epoch 4/100
1185/1185 - 10s - loss: 0.0108 - accuracy: 0.8650 - val_loss: 0.0143 - val_accuracy: 0.7248 - 10s/epoch - 8ms/step
Epoch 5/100
1185/1185 - 10s - loss: 0.0105 - accuracy: 0.8786 - val_loss: 0.0149 - val_accuracy: 0.7153 - 10s/epoch - 8ms/step
Epoch 6/100
1185/1185 - 10s - loss: 0.0102 - accuracy: 0.8918 - val_loss: 0.0108 - val_accuracy: 0.8647 - 10s/epoch - 8ms/step
Epoch 7/100
1185/1185 - 10s - loss: 0.0100 - accuracy: 0.8989 - val_loss: 0.0149 - val_accuracy: 0.7405 - 10s/epoch - 8ms/step
Epoch 8/100
1185/1185 - 10s - loss: 0.0099 - accuracy: 0.9021 - val_loss: 0.0106 - val_

In [37]:
def build_NeuralNetwork():

    #input layer
    model = Sequential()
    model.add(Input(shape=(100,100,3)))
    model.add(Flatten())

    #hiddenlayer 1
    model.add(Dense(10))
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))

    #hiddenlayer 2
    model.add(Dense(100)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 3
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #hiddenlayer 4
    model.add(Dense(300)) #900
    model.add(BatchNormalization())
    model.add(Activation(activation='sigmoid'))
    model.add(Dropout(0.25))

    #output layer
    model.add(Dense(131,activation="softmax"))

    model.compile(loss = 'poisson', optimizer = "adam", metrics = ['accuracy'])
    print('Compiled!')

    return model

In [38]:
network = build_NeuralNetwork()
print('Model built.')
modelname = 'ann2'
checkpointer = ModelCheckpoint(filepath = modelname+'.hdf5', verbose = 0, save_best_only = True)

history = network.fit(train_images_array, train_hot_class,
    batch_size = 32, #32
    epochs = 100, #15
    validation_data = (valid_images_array, valid_hot_class),
    verbose = 2,
    callbacks=[checkpointer],
    shuffle = True)

network.save('model_'+modelname+'.h5')

train_loss, train_acc = network.evaluate(train_images_array, train_hot_class, verbose=0)
print('Restored model, train accuracy: {:5.2f}%'.format(100 * train_acc))
valid_loss, valid_acc = network.evaluate(valid_images_array, valid_hot_class, verbose=0)
print('Restored model, valid accuracy: {:5.2f}%'.format(100 * valid_acc))
new_test_loss, new_test_acc = network.evaluate(new_test_images_array, new_test_hot_class, verbose=0)
print('Restored model, new_test accuracy: {:5.2f}%'.format(100 * new_test_acc))
test_loss, test_acc = network.evaluate(test_images_array, test_hot_class, verbose=0)
print('Restored model, test accuracy: {:5.2f}%'.format(100 * test_acc))
np.save(modelname+'_history.npy',history.history)

Compiled!
Model built.
Epoch 1/100
1185/1185 - 22s - loss: 0.0314 - accuracy: 0.2445 - val_loss: 0.0230 - val_accuracy: 0.4685 - 22s/epoch - 19ms/step
Epoch 2/100
1185/1185 - 10s - loss: 0.0194 - accuracy: 0.5655 - val_loss: 0.0201 - val_accuracy: 0.4587 - 10s/epoch - 9ms/step
Epoch 3/100
1185/1185 - 10s - loss: 0.0156 - accuracy: 0.6940 - val_loss: 0.0153 - val_accuracy: 0.6775 - 10s/epoch - 8ms/step
Epoch 4/100
1185/1185 - 11s - loss: 0.0139 - accuracy: 0.7511 - val_loss: 0.0171 - val_accuracy: 0.6056 - 11s/epoch - 9ms/step
Epoch 5/100
1185/1185 - 11s - loss: 0.0129 - accuracy: 0.7812 - val_loss: 0.0109 - val_accuracy: 0.8761 - 11s/epoch - 10ms/step
Epoch 6/100
1185/1185 - 15s - loss: 0.0122 - accuracy: 0.8112 - val_loss: 0.0113 - val_accuracy: 0.8512 - 15s/epoch - 13ms/step
Epoch 7/100
1185/1185 - 12s - loss: 0.0117 - accuracy: 0.8269 - val_loss: 0.0124 - val_accuracy: 0.7905 - 12s/epoch - 10ms/step
Epoch 8/100
1185/1185 - 12s - loss: 0.0113 - accuracy: 0.8445 - val_loss: 0.0103 - v

In [ ]:

history2=np.load('my_history.npy',allow_pickle='TRUE').item()


In [ ]:
plt.plot(history2['accuracy'])

In [ ]:
new_model = tf.keras.models.load_model('model_ann1.h5')
new_model.summary()

Model accuracy

In [ ]:
plt.figure(figsize = (6, 5))
plt.plot(history.history['accuracy'], color = 'r')
plt.plot(history.history['val_accuracy'], color = 'b')
plt.title('Model Accuracy', weight = 'bold', fontsize = 16)
plt.ylabel('accuracy', weight = 'bold', fontsize = 14)
plt.xlabel('epoch', weight = 'bold', fontsize = 14)
plt.ylim(0.6, 1.1)
plt.xticks(weight = 'bold', fontsize = 12)
plt.yticks(weight = 'bold', fontsize = 12)
plt.legend(['train', 'val'], loc = 'lower right', prop = {'size': 14})
plt.grid(color = 'y', linewidth = '0.5')
plt.show()

Show prediction.

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(1)  
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'lower right')
   
 # summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()